In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
service_credential = dbutils.secrets.get(scope="databricksscopename",key="databricksservicekey")

spark.conf.set("fs.azure.account.auth.type.adlsx5u224gv3xgd2.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.adlsx5u224gv3xgd2.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.adlsx5u224gv3xgd2.dfs.core.windows.net", "5c962712-e111-48b0-b9af-41f03175e216")
spark.conf.set("fs.azure.account.oauth2.client.secret.adlsx5u224gv3xgd2.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.adlsx5u224gv3xgd2.dfs.core.windows.net", "https://login.microsoftonline.com/72f988bf-86f1-41af-91ab-2d7cd011db47/oauth2/token")

In [ ]:
schema = ArrayType(
    StructType(
        [
            StructField("qty", IntegerType(), True),
            StructField("unit", StringType(), True),
            StructField("curr", StringType(), True),
            StructField("id", StringType(), True),
            StructField("name", StringType(), True),
            StructField("price", IntegerType(), True),
            StructField(
                "promotion_info",
                StructType(
                    [
                        StructField("promo_id", IntegerType(), True),
                        StructField("promo_qty", IntegerType(), True),
                        StructField("promo_disc", DecimalType(3, 2), True),
                        StructField("promo_item", StringType(), True),
                    ]
                ),
                True,
            ),
        ]
    )
)

schema2 = ArrayType(
    StructType(
        [
            StructField("promo_id", IntegerType(), True),
            StructField("promo_qty", IntegerType(), True),
            StructField("promo_disc", DecimalType(3, 2), True),
            StructField("promo_item", StringType(), True),
        ]
    ),
    True,
)
schema3 = ArrayType(ArrayType(StringType(), True), True)

df1 = spark.read \
        .format("delta") \
        .load("dbfs:/user/hive/warehouse/retail_org_dt.db/sales_orders_raw") \
        .select("*") \
        .withColumn("clicked_items", from_json(col("clicked_items"), schema3)) \
        .withColumn("promo_info", from_json(col("promo_info"), schema2)) \
        .withColumn("ordered_products", from_json(col("ordered_products"), schema)) \
        .withColumn("ordered_products", explode("ordered_products")) \
        .withColumn("order_datetime", from_unixtime("order_datetime")) \
        .withColumn("product_id", col("ordered_products").id) \
        .withColumn("unit_price", col("ordered_products").price) \
        .withColumn("quantity", col("ordered_products").qty)
        
df1.write.format("delta").saveAsTable("retail_org_dt.sales_orders_cleansed")

In [ ]:
df2 = spark.read \
        .format("delta") \
        .load("dbfs:/user/hive/warehouse/retail_org_dt.db/customers_raw") \
        .select("*") \
        .withColumn("customer_id",col("customer_id").cast("integer")) \
        .withColumn("lon",col("lon").cast("double")) \
        .withColumn("lat",col("lat").cast("double")) \
        .withColumn("loyalty_segment",col("loyalty_segment").cast("integer"))

df2.write.format("delta").saveAsTable("retail_org_dt.customers_cleansed")

In [ ]:
df3 = spark.read \
        .format("delta") \
        .load("dbfs:/user/hive/warehouse/retail_org_dt.db/products_raw") \
        .select("*") \
        .withColumn("ean13",col("ean13").cast("double"))

df3.write.format("delta").saveAsTable("retail_org_dt.products_cleansed")